# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create  Keyspace 

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """
)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above

# Set session row_factory so that select results could be read into a dataframe.
# Copied from https://stackoverflow.com/questions/41247345/python-read-cassandra-data-into-pandas
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

try:
    session.set_keyspace('udacity')
    session.row_factory = pandas_factory
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

# DATA MODEL DESIGN NOTES:
# We will call table session_activity and Primary_Key should be partitioned by session_Id 
# and add item_In_Session as clustering key. So rows are first partitioned by sessionId and then 
# ordered by item_In_Session within that partition.
# Columns = session_id, item_in_session, artist, song, length
# select artist, song, length from session_activity where sessionId = 338 and itemInSession = 4

query = "DROP TABLE IF EXISTS session_activity"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "CREATE TABLE IF NOT EXISTS session_activity "
query = query + "(session_id INT, item_in_session INT, artist TEXT, song TEXT, length FLOAT, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [9]:
## INSERT into session_activity. Part of the code to set up the CSV file provided. complete the Apache Cassandra code#
## See images/image_event_datafile_new.jpg for order in which values appear in line.

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_activity (session_id, item_in_session, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## SELECT statement to verify the data was entered into the table
query = "SELECT artist, song, length FROM session_activity WHERE session_id = 338 AND item_in_session = 4"

try:
    rows = session.execute(query)
    df = rows._current_rows
except Exception as e:
    print(e)
    
df

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

# DATA MODEL DESIGN NOTES:
# We will call this table user_activity and Primary_Key should be partitioned by user_id and session_id 
# and add item_In_Session as clustering key. So rows are first partitioned by user_id and sessionId and then 
# ordered by item_In_Session.
# Columns = user_id,session_id, item_in_session, artist, song, first_name, last name
# select artist, song, first_name, last_name from user_activity WHERE user_id = 10 and session_id = 182

query = "DROP TABLE IF EXISTS user_activity"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "CREATE TABLE IF NOT EXISTS user_activity "
query = query + "(user_id int, session_id int, item_in_session int, artist text, song text, first_name text, last_name text, " 
query = query + " PRIMARY KEY (user_id, session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
## INSERT rows into user_activity from event_datafile_new.csv
## See images/image_event_datafile_new.jpg for order in which values appear in line.

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_activity (user_id, session_id, item_in_session, artist, song, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [13]:
## SELECT statement to verify the data was entered into the table
query = "SELECT artist, song, first_name, last_name FROM user_activity WHERE user_id = 10 AND session_id = 182"
try:
    rows = session.execute(query)
    df = rows._current_rows
except Exception as e:
    print(e)
    
df

,artist,song,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


In [14]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# DATA MODEL DESIGN NOTES:
# We will call this table song_activity and Primary_Key should be partitioned by song
# and add user_id as clustering key. So rows are first partitioned by song  
# ordered by user_id within partition.
# Columns = song,user_id,first_name,last name
# select first_name, last_name from song_event WHERE song = 'All Hands Against His Own'

query = "DROP TABLE IF EXISTS song_activity"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "CREATE TABLE IF NOT EXISTS song_activity "
query = query + "(song text, user_id int, first_name text, last_name text," 
query = query + " PRIMARY KEY (song, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)
        

In [15]:
## INSERT rows into song_activity from event_datafile_new.csv
## See images/image_event_datafile_new.jpg for order in which values appear in line.

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_activity (song, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [16]:
## SELECT statement to verify the data was entered into the table
query = "SELECT first_name, last_name FROM song_activity WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
    df = rows._current_rows
except Exception as e:
    print(e)

df

,first_name,last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [17]:
## Drop the table before closing out the sessions
query = "DROP TABLE IF EXISTS session_activity"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS user_activity"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS song_activity"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()